<a target="_blank" href="https://colab.research.google.com/github/juanhuguetgarcia/intro_to_llms/blob/main/intro_to_llms/2_langgraph_our_calculator_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# 🧑‍🏫 Tutorial: Building Your First AI Agent with LangGraph, LangChain & Pydantic

Here’s a step-by-step interactive guide to understand how to build our first AI agent using:

* 🧠 LangChain for agent logic
* 🔁 LangGraph for orchestration
* 📦 Pydantic for clean state structure
* 🧮 Calculator tool for real functionality

In this notebook, we’re going to build a simple AI agent that can answer math questions like:

“What is 24 * 3?”

We’ll do it step by step. Here’s what we’re going to do:

## 🧱 Step 1: Define What Our Agent Needs to Know (the “state”)

We’ll create a little box of information (called state) where we put:
* the question we ask (input)
* the answer the agent gives us (output)

To keep things clean and organized, we’ll use a little helper called Pydantic.
Think of it like a smart checklist that makes sure our box always has the right items.

## 🔢 Step 2: Create a Typed Calculator Tool Using @tool

Instead of passing in a string like "24 * 3" and parsing it with eval, we’re going to build a real Python function that accepts two numbers and returns the result.

LangChain allows us to turn any function into an AI tool using a special decorator called @tool.

## 🤖 Step 3: Build the Agent

We’ll connect the tool to an AI model using LangChain. The AI will be smart enough to:
* read the question,
* choose the calculator if needed,
* return the answer.

This is the brain of our agent.

## 🔄 Step 4: Create a Graph (Flow)

We’ll then tell LangGraph:
	•	where to start,
	•	what steps to follow,
	•	and where to end.

This is like drawing a simple flowchart:
→ Get the question → Use the agent → Return the answer

## ▶️ Step 5: Run It!

Finally, we’ll give it a question like "What is 12 * 6?", and watch the agent respond with the right answer.

# Install required packages

Packages required: langgraph, langchain_openai and langchain_core

In [ ]:
# Install LangGraph and langchain packages
%pip install --quiet -U langgraph langchain_openai langchain_core python-dotenv pydantic

## Set Open AI Key

Set the api key in the .env variable

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import os
print(os.environ.get("OPENAI_API_KEY")[:10] + "...")

### 🧱 Step 1: Define What Our Agent Needs to Know (the “state”)
We’ll create a little box of information (called state) where we put:

* the question we ask (input_msg)

* the answer the agent gives us (output_msg)

To keep things clean and organized, we’ll use a little helper called Pydantic. Think of it like a smart checklist that makes sure our box always has the right items.

🧠 Why Pydantic?
* Provides type safety and autocomplete in editors
* Ensures clean, validated state
* Avoids messy dicts and typos
* **LangGraph supports it natively for StateGraph**

In [ ]:
from pydantic import BaseModel
from typing import Union

class AgentState(BaseModel):
    input: ...
    output: ... = ""  # this will be updated after tool execution

We can check that pydantic is giving us a nice representation...

In [ ]:
# pass input and output strings to the AgentState model

print(AgentState(...).model_dump_json(indent=2))

### 🔢 Step 2: Create a Typed Calculator Tool Using @tool

Instead of passing in a string like "24 * 3" and parsing it with eval, we’re going to build a real Python function that accepts two numbers and returns the result.

LangChain allows us to turn any function into an AI tool using a special decorator called @tool

🧠 Why Use @tool?

Using @tool is:
* 🔐 Safe — no risky parsing or code execution like eval
* 🧼 Clean — LangChain will automatically read:
- the function name (calculate)
- the parameter names and types (a: int, b: int)
- the docstring to describe the tool
- 🤖 Agent-friendly — the agent will now know exactly how to use this tool: “If I get a question like ‘What is 5 plus 7?’, I can call calculate(a=5, b=7).”

In [ ]:
from langchain.tools import tool
@tool
def calculate(a: int, b: int) -> str:
    """..."""
    return str(...)

We can see how it actually it adds a pydantic model with the metadata...

In [ ]:
type(calculate.args_schema)

In [ ]:
calculate.args_schema.model_json_schema()

In [ ]:
calculate.args

### 🤖 Step 3: Build the Agent

Now that we have a proper tool defined, let’s create the agent that knows how to use it.

🧠 What’s Going On Here?

Initialize_agent sets up a reasoning agent that knows how to:

* Interpret your question
* Decide whether a tool is needed
* Call that tool with the right parameters
* Return the final answer
* `AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` is perfect for functions with multiple inputs:
    * It uses tool descriptions and the prompt itself to choose actions
    * No need to hard-code logic

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai.chat_models import ChatOpenAI

# Create the LLM (you can replace with any compatible model)
llm = ChatOpenAI(model="...", temperature=...)

# Initialize the agent with our tool
agent_executor = initialize_agent(
    tools=[..., ],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [ ]:
agent_executor.invoke(...)

### 🧠 Step 5: Create a Node That Runs the Agent

In LangGraph, we wrap each step in a function that takes the current state, does something, and returns the updated state.

💡 Reminder: What’s “State”?

Think of AgentState as a box that carries:
* input: what the user asked
* output: what the agent answered

Each LangGraph node receives the box, modifies it, and passes it on.

In [ ]:
## we have defined the AgentState above

# Node that runs the agent. This is a langchain runnable, so it could be as well the RAG we did last session
def run_agent(state: AgentState) -> AgentState:
    result = agent_executor.run(state.input)
    return AgentState(input=state.input, output=result)

## Create the Graph

Create a node that calls LLM

As there are no other nodes, we will connect the START and the END to this node and compile.

## Understanding LangGraph Components

🧠 Why Use a Graph Here?

LangGraph lets us define structured flows:

* In this example, it’s a single-node graph
* Later, we can add conditional branches, tools, memory, retries, etc.
* It also gives us visualization and execution tracing for free


In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

# Add our only node: the agent itself
graph.add_node("agent", run_agent)

# Define the flow: start at the agent, then end
graph.set_entry_point("agent")
graph.add_edge("agent", END)

# Compile it
app = graph.compile()

## Visualize the graph

Visualize the graph using display() method

In [ ]:
# Visualize your graph
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

# Test

Test the graph using app.invoke() method.


In [ ]:
app.invoke({"input": "What is 13 plus 29?"})

## ✅ Recap

You just built a real AI agent that can:
* Understand natural language
* Choose a Python tool to use
* Return accurate results

You used:
* Pydantic to structure state
* LangChain to build the agent
* LangGraph to control the flow

## Bonus

Let’s add a second tool and introduce branching logic in the graph so your students can learn how LangGraph routes input based on content.

## 🎯 Goal

* Add a second tool: get_time() → returns the current time.
* Add a classifier node that decides:
    * If the question is about math → use the calculator agent
    * If the question is about time → use the time tool

Use add_conditional_edges to route input based on type

In [ ]:
from langchain.tools import tool
from datetime import datetime

@tool
def get_time() -> str:
    """Returns the current time."""
    return datetime.now().strftime("%H:%M:%S")

## 🧠 2. Update Agent Tool List

Update the agent with both tools (note: only calculate needs structured input):

In [ ]:
agent_executor = initialize_agent(
    tools=[calculate, get_time],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

## 🧠 3. Define a Classifier Node

This node routes inputs to the correct node:

In [ ]:
class AgentStateBranch(BaseModel):
    input: str
    output: str = ""
    branch: Literal["math","time"] = "math"

In [ ]:
from typing import Literal

def classify_question(state: AgentStateBranch) -> AgentStateBranch:
    text = state.input.lower()
    if "time" in text:
        branch = "time"
    else:
        branch = "math"
    return AgentState(
      input=state.input,
      output=state.output,
      branch=branch
    )

### ⚙️ 4. Define a Time Tool Node

We wrap the get_time() tool into a node:

In [ ]:
def run_time_tool(state: AgentStateBranch) -> AgentStateBranch:
    result = get_time.invoke({})
    return AgentState(input=state.input, output=result)

### Rebuild the graph

In [ ]:
from langgraph.graph import StateGraph, START, END

# Build the LangGraph with Branching
graph = StateGraph(AgentStateBranch)

# Nodes
graph.add_node("classify", classify_question)
graph.add_node("math_agent", run_agent)
graph.add_node("time_tool", run_time_tool)

# Flow
graph.add_edge(START, "classify")  # ✅ Kick off at START
graph.add_conditional_edges("classify", lambda s: s.branch, {
    "math": "math_agent",
    "time": "time_tool"
})

graph.add_edge("math_agent", END)
graph.add_edge("time_tool", END)

# Compile
app = graph.compile()

In [ ]:
# Visualize your graph
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
print(app.invoke({"input": "What is 6 + 9?"}))

In [ ]:
print(app.invoke({"input": "What time is it ?"}))